<a href="https://colab.research.google.com/github/Trickshotblaster/nn-practices/blob/main/TrigramLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [448]:
import random
import math

In [449]:
import requests

url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
url = 'https://raw.githubusercontent.com/karpathy/makemore/master/names.txt'
#url = 'https://raw.githubusercontent.com/Trickshotblaster/nn-practices/main/TrigramLM.ipynb'
response = requests.get(url)

with open('file.txt', 'wb') as f:
  f.write(response.content)

In [450]:
with open('file.txt', 'r') as data:
  dataset = data.readlines()

with open('file.txt', 'r') as data:
  raw_data = data.read()


In [451]:
dataset = raw_data.split("\n")

In [452]:
dataset[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [453]:
for i, data in enumerate(dataset):
  dataset[i] = "." + data + "."

In [454]:
dataset[:10]

['.emma.',
 '.olivia.',
 '.ava.',
 '.isabella.',
 '.sophia.',
 '.charlotte.',
 '.mia.',
 '.amelia.',
 '.harper.',
 '.evelyn.']

In [455]:
raw_data = ''.join(dataset)

In [456]:
raw_data[:10]

'.emma..oli'

In [457]:
train_set = dataset[:math.floor(len(dataset) * 0.8)]
dev_set = dataset[math.floor(len(dataset) * 0.8):math.floor(len(dataset) * 0.9)]
test_set = dataset[math.floor(len(dataset) * 0.9):len(dataset) - 1]

In [458]:
train_set[:10]

['.emma.',
 '.olivia.',
 '.ava.',
 '.isabella.',
 '.sophia.',
 '.charlotte.',
 '.mia.',
 '.amelia.',
 '.harper.',
 '.evelyn.']

In [459]:
dev_set[:10]

['.amrom.',
 '.aniket.',
 '.ansen.',
 '.apolo.',
 '.aqib.',
 '.aquarius.',
 '.araf.',
 '.arafat.',
 '.areeb.',
 '.ariez.']

In [460]:
test_set[:10]

['.arib.',
 '.arinze.',
 '.aristeo.',
 '.arlind.',
 '.armahni.',
 '.arnez.',
 '.arnie.',
 '.arrie.',
 '.arris.',
 '.arrison.']

In [461]:
vocab = set(raw_data)

In [462]:
vocab

{'.',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [463]:
vocab_len = len(vocab)

In [464]:
vocab_len

27

In [465]:
stoi = {char:i for i, char in enumerate(vocab)}

In [466]:
itos = {i:char for i, char in enumerate(vocab)}

In [467]:
import torch

In [468]:
import torch.nn.functional as functional

In [469]:
# make dataset
xs, ys = [], []
for data in train_set:
  for ch1, ch2, ch3 in zip(data, data[1:], data[2:]):
    xs.append([stoi[ch1], stoi[ch2]])
    ys.append(stoi[ch3])

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
num

314304

In [470]:
ws = torch.randn((vocab_len*2, vocab_len))
ws.requires_grad = True
ws.shape

torch.Size([54, 27])

In [471]:
xenc = functional.one_hot(xs[0], num_classes=vocab_len).float().flatten()
xenc @ ws

tensor([-0.4084,  0.3408,  1.7516, -0.7858,  0.3800,  0.3164,  0.2218, -1.1666,
        -1.2482,  3.7461, -1.8359,  0.0464,  1.4693, -1.5752, -0.7963, -2.6585,
        -0.5314,  0.9454,  0.7938, -1.4467, -0.5844, -2.4619, -1.1767,  0.1932,
         2.3272, -0.0392,  1.5166], grad_fn=<SqueezeBackward4>)

In [472]:
(xenc @ ws).shape

torch.Size([27])

In [473]:
num_epochs = 100
lr = 20

In [474]:
torch.randn((3, 1))

tensor([[-0.6523],
        [ 0.2456],
        [ 0.5307]])

In [475]:
for epoch in range(num_epochs):

    xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
    logits = xenc @ ws
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdim=True)
    loss = -probs[torch.arange(int(num / 2)), ys].log().mean() + (ws.mean() * 0.01)
    print(loss.item())

    ws.grad = None
    loss.backward()
    ws.data += ws.grad * -lr

3.9720306396484375
3.67653489112854
3.4562888145446777
3.2916576862335205
3.1641571521759033
3.0605287551879883
2.9741897583007812
2.9016494750976562
2.8406293392181396
2.78926157951355
2.7457776069641113
2.7085485458374023
2.676234006881714
2.647834300994873
2.6226298809051514
2.6000959873199463
2.579831123352051
2.5615177154541016
2.544893741607666
2.529743194580078
2.515883445739746
2.5031614303588867
2.4914438724517822
2.4806182384490967
2.4705874919891357
2.4612672328948975
2.4525833129882812
2.444471836090088
2.4368762969970703
2.429748296737671
2.423043727874756
2.416724443435669
2.410756826400757
2.4051103591918945
2.399759531021118
2.394679307937622
2.3898489475250244
2.385249376296997
2.380863904953003
2.376676321029663
2.372673988342285
2.368842363357544
2.3651719093322754
2.3616509437561035
2.3582701683044434
2.3550214767456055
2.3518965244293213
2.3488876819610596
2.3459887504577637
2.343193292617798
2.340496063232422
2.3378913402557373
2.335374116897583
2.332940101623535


In [476]:
xs[:10]

tensor([[12, 21],
        [21, 20],
        [20, 20],
        [20,  9],
        [12, 18],
        [18,  3],
        [ 3, 25],
        [25,  1],
        [ 1, 25],
        [25,  9]])

In [477]:
def forward(xs):
  xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
  logits = xenc @ ws
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdims=True)
  outchar = itos[torch.multinomial(probs, num_samples=1).item()]
  return outchar

def prompt(ins):
  out = ins
  for char in range(500):
    last_two = ins[-2:]
    xs = torch.tensor([[stoi[last_two[0]], stoi[last_two[1]]]])
    outs = forward(xs)
    out += outs
  return out

prompt("Rom")

'Romda.eiiaa.ayqsieii.ii.jaeauaiib.pviieiaii.a.oioeaaabeiiiawmiueeareeawiaaalibl.eeaeaeaaaxeiiya.iabiu.ae.ageeyaaibsmavmaaieaaiaaymeaafaaifaioiixaaigiiyeeiiei.eaaaazybiaarewaaa..bdb.aa.yaieaaaeaaiiaaaeaaiaaaf.aaiay.eaijiieaeyaafazeb.ae..aiiaaeiae.igaaeabaaaiav.h.caeau.bjiaaabii.a.vbie.aei.iabuiae.ua.ene.anai...t.iaiueaiba.raiaai.nyiiiieaataaab.aiuaai.alibeiaaaaa.iwkbaaiina.yaeae..aiibbeaa.ie.aiamie.aaoai.b.bawei.yareaebeebveeaaaiai.yiea.iihaia.azniueaeaaiaeia.aiy..aaeaf.icaeiaebi.iafyiaawi.aaiaebaid'

In [478]:
# make test set
txs, tys = [], []
for data in test_set:
  for ch1, ch2, ch3 in zip(data, data[1:], data[2:]):
    txs.append([stoi[ch1], stoi[ch2]])
    tys.append(stoi[ch3])

txs = torch.tensor(txs)
tys = torch.tensor(tys)
tnum = txs.nelement()
tnum

39052

In [479]:
xenc = functional.one_hot(txs, num_classes=vocab_len).float().flatten(start_dim=1)
logits = xenc @ ws
counts = logits.exp()
probs = counts / counts.sum(1, keepdim=True)
loss = -probs[torch.arange(int(tnum / 2)), tys].log().mean() + (ws.mean() * 0.01)
print(loss.item())

2.487349033355713


In [480]:
# make dev set
dxs, dys = [], []
for data in dev_set:
  for ch1, ch2, ch3 in zip(data, data[1:], data[2:]):
    dxs.append([stoi[ch1], stoi[ch2]])
    dys.append(stoi[ch3])

dxs = torch.tensor(dxs)
dys = torch.tensor(dys)
dnum = dxs.nelement()
dnum

38860

In [481]:
dxs[:10]

tensor([[12,  9],
        [ 9, 20],
        [20, 19],
        [19, 18],
        [18, 20],
        [12,  9],
        [ 9,  0],
        [ 0, 25],
        [25, 24],
        [24, 21]])

In [482]:
xenc = functional.one_hot(dxs, num_classes=vocab_len).float().flatten(start_dim=1)
logits = xenc @ ws
counts = logits.exp()
probs = counts / counts.sum(1, keepdim=True)
loss = -probs[torch.arange(int(dnum / 2)), dys].log().mean() + (ws.mean() * 0.01)
print(loss.item())

2.4953885078430176


Looks like it might be overfitting? The dev set results seem fine, but the test set loss it much higher than it should be.

In [483]:
ws = torch.randn((vocab_len*2, vocab_len))
ws.requires_grad = True
ws.shape

torch.Size([54, 27])

In [484]:
num_epochs = 100
lr = 20
regularization = 0.01

In [485]:
last_loss = 2.1

for epoch in range(num_epochs):
    dxenc = functional.one_hot(dxs, num_classes=vocab_len).float().flatten(start_dim=1)
    dlogits = dxenc @ ws
    dcounts = dlogits.exp()
    dprobs = dcounts / dcounts.sum(1, keepdim=True)
    dloss = -dprobs[torch.arange(int(dnum / 2)), dys].log().mean() + (ws.mean() * regularization)
    dloss.detach()

    xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
    logits = xenc @ ws
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdim=True)
    loss = -probs[torch.arange(int(num / 2)), ys].log().mean() + (ws.mean() * regularization * (abs(last_loss - dloss)))

    print(loss.item())
    ws.grad = None
    loss.backward()
    ws.data += ws.grad * -lr

    last_loss = loss.item()

3.9933948516845703
3.6653103828430176
3.4273719787597656
3.254211902618408
3.125101089477539
3.024634838104248
2.943120002746582
2.8754098415374756
2.818420648574829
2.769979476928711
2.72843074798584
2.6924736499786377
2.6610751152038574
2.633422613143921
2.608877658843994
2.5869436264038086
2.567228317260742
2.5494189262390137
2.533259391784668
2.518538236618042
2.505075693130493
2.492720365524292
2.4813413619995117
2.4708266258239746
2.4610798358917236
2.452017307281494
2.4435675144195557
2.4356679916381836
2.42826509475708
2.4213125705718994
2.4147684574127197
2.408597707748413
2.402768611907959
2.3972535133361816
2.392026662826538
2.3870673179626465
2.382354497909546
2.3778703212738037
2.37359881401062
2.369525194168091
2.3656363487243652
2.361919403076172
2.358363628387451
2.3549585342407227
2.3516948223114014
2.3485636711120605
2.3455569744110107
2.342667579650879
2.339888572692871
2.3372130393981934
2.3346359729766846
2.3321516513824463
2.3297548294067383
2.3274407386779785
2.3

In [486]:
prompt("bro")

'broenn.zchcanegsnsalbn..kil..ncnlnssnonenn.isnepvn.nxpsnnsy...swsaspnnbkios.ocoouih.lsnisscnnln.ls.tsnsnspnnony.ssnsqdcndyhlhnonyenrpnsysuneenogei.sunnp.mcnshsnoe.eh.nnmsnnbdknunn.nnsnsut..cgn.knsr.ylh.dnwnllc.rn.goknssrznnldneuneep.dgckn.oncd.s.chsnzqsennvssrnnz.nnt.ognsgnuyoknssvslsnnonmcsnnnrssynednvnnnnnnn.snlsybgnionkid.asns.nlno.nn.snnhnsiilavh.ssnl.onnesn.dqke..nosdsun.snnnlsano.snsn.jd.pslnrnonngyoanngnsnlnusnacols.nlyshnnnnsnnho.nnduasnvssneenszlshsnnthine..segsndrnennuhnynv.o.e.dwnnrnnskn'

Idk, maybe? I've definitely screwed something up here but I have no idea what

In [487]:
num_epochs = 100
lr = 20
regularization = 0.1

In [488]:
ws = torch.randn((vocab_len*2, vocab_len))
ws.requires_grad = True
ws.shape

torch.Size([54, 27])

In [489]:
for epoch in range(num_epochs):

    xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
    logits = xenc @ ws
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdim=True)
    loss = -probs[torch.arange(int(num / 2)), ys].log().mean() + (ws.mean() * regularization)

    print(loss.item())
    ws.grad = None
    loss.backward()
    ws.data += ws.grad * -lr



4.29519510269165
3.8420252799987793
3.5212149620056152
3.293205976486206
3.1379246711730957
3.0283141136169434
2.9440038204193115
2.8752810955047607
2.8177661895751953
2.7690062522888184
2.727297782897949
2.6913259029388428
2.6600441932678223
2.6326217651367188
2.608396291732788
2.586840867996216
2.567530870437622
2.5501203536987305
2.5343313217163086
2.5199341773986816
2.5067412853240967
2.494598627090454
2.4833784103393555
2.4729716777801514
2.4632883071899414
2.454249382019043
2.445789098739624
2.4378485679626465
2.43037748336792
2.423332929611206
2.4166760444641113
2.4103732109069824
2.4043943881988525
2.3987138271331787
2.3933069705963135
2.3881547451019287
2.383236885070801
2.3785369396209717
2.374040365219116
2.3697333335876465
2.365602970123291
2.361637830734253
2.3578286170959473
2.3541648387908936
2.3506386280059814
2.3472414016723633
2.343966245651245
2.340806484222412
2.3377554416656494
2.334808111190796
2.331958293914795
2.3292012214660645
2.3265328407287598
2.323948144912

In [490]:
prompt('em')

'em.aoaii.aj...aiye.aaoi.aaii..aiaii.eiiiet.uyiiaaia.g..ai.baraaiyauimy.e.lmam.a..iaaa.iruqeliaoam.saiaea.aaeibya.i.iiema.eii.aaoabaaayaeatiaayaufh.iaeaai.omauie.kaaialyaaa.bi.a.oyxaya.aiamioty.aaai..tem..mmaee.i.s..aaiiia.i.adti.alii.smaioaimbjii.aa.iei.aiaaey.ai.ocbmuyuy.ayeeilie.qa.i.iaiaiitsaiym.i.ai.ime.yay.aamiii.i.mmai.eaie.ytieioariqaaemi.e.a.iiy.ai.iiia.e.ya.ia.ii.amaaemiusonyeaeai.aema.uwdieyialbaseaaieeer.iamaa.aaai.oiacaiaza.aaeaaao.yaiataimit..a.rtsam..iame.ahiaeaia.amahidiaeeaioag.a.j'

Okay this seems promising...

aeoioa, vim, esiaota, aipe, seto, and ioa are all decent

In [491]:
prompt('em')

'emcaey.yai.i.ieaaltamiaas.eouaaaiyii.b.ayii.eim.iyy.o.aaialie.yr.ieailanojiii.aiaiee.tay.eayitii.aii.yna.aaayanmaa.oiali.itcayeal.ha.ualm..eema.deeia.a.iaema.i.biaggaitiaaaa.oguv.a..icy.r..tig.e.y.aahaila.ioiaihai.ei.ioiai.e.a..am.oca..ma.iaibaaayaioaaeamac.eiaa.f.aea...daiooi.mymtviearmat.a.aiyaamqoaiaiajagjaa..eciai.a.mi.ataya.tytaaaa..aeaatbamal.oaaaidmd..apeaawaj.yit.iiiiei.a.snyaayaa.iaino.arimayraaa..hii.rayti.aiiaaaiaigih.iel.as.a.eeioaia.aieoeaae..iiiloga.iyy..bt.aeom.aeabmdooeaaiaaieaaian'

In [492]:
ws = torch.randn((vocab_len*2, vocab_len))
ws.requires_grad = True
ws.shape

torch.Size([54, 27])

In [493]:
prompt('em')

'embmqqbbbbqnq.abnqhbtbb..tqbbbbbnq.bb.hbbbbqqqbbb.bbqqbrqy..obbq.bybhhbhwqnbbbbbbboymhbbbbbbbbbbbbbbbb.hbbybb.bnbbbbbbhbbbbbmq.bbbbb.babqpbbbbbbbbqbbq.bnbbhqbbpq.bbbbbqbbtbbbbtbbqtbab.bbyhb.bbbbbbgbqbbb.qmbbbqqbqbb.qbbqbebqbbq.qqbqbymqb.bbqqzbbbbqbqby.fbzhbbbbqsaobbh.qbbbbbbbqbbqbqby.bbbbbhybbbbqbbbbbbqbbhbbbbsqbbbq.bqqyzbbbbbbb.bbsqhbqbbbqbqbbfbbqbbbpbbbbbbbbbbbqbbhbbnwbbykbqbtblbqbbbbbqbyqbb.qbqbbqbnbb..by.fhbbqqbbhbxbzqbbyqnbqbb.qbbbqbbbbbyqbbbrbbbbbbqqhbbbbnhbbbmqqbbbqbbnbyqbbbbxb.qbbqnbqqqbhg'

There's definitely some improvement over base randomness sometimes, but for others, it sucks

In [494]:
regularization = 0.05

In [495]:


for epoch in range(num_epochs):
    dxenc = functional.one_hot(dxs, num_classes=vocab_len).float().flatten(start_dim=1)
    dlogits = dxenc @ ws
    dcounts = dlogits.exp()
    dprobs = dcounts / dcounts.sum(1, keepdim=True)
    dloss = -dprobs[torch.arange(int(dnum / 2)), dys].log().mean() + (ws.mean() * regularization)
    dloss.detach()

    xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
    logits = xenc @ ws
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdim=True)
    loss = -probs[torch.arange(int(num / 2)), ys].log().mean() + (ws.mean() * regularization)

    print(loss.item(), dloss.item())
    ws.grad = None
    loss.backward()
    ws.data += ws.grad * -lr


4.355617046356201 4.329306602478027
3.924482583999634 3.953385829925537
3.5875072479248047 3.6562716960906982
3.336085319519043 3.432685375213623
3.161936044692993 3.2779321670532227
3.042645215988159 3.173408031463623
2.953641891479492 3.0967767238616943
2.881988286972046 3.0359373092651367
2.8222384452819824 2.985684394836426
2.7715303897857666 2.9432756900787354
2.7280261516571045 2.907015085220337
2.690399408340454 2.875734806060791
2.657621383666992 2.8485476970672607
2.6288671493530273 2.8247411251068115
2.603475570678711 2.80373477935791
2.580916404724121 2.7850584983825684
2.560764789581299 2.76833176612854
2.5426735877990723 2.7532474994659424
2.5263559818267822 2.7395544052124023
2.5115702152252197 2.727046489715576
2.498110294342041 2.715553045272827
2.4858012199401855 2.7049336433410645
2.4744937419891357 2.6950738430023193
2.464062452316284 2.685879707336426
2.4544003009796143 2.6772751808166504
2.4454185962677 2.6691970825195312
2.437042474746704 2.6615936756134033
2.4292

In [496]:
dxs[:10]

tensor([[12,  9],
        [ 9, 20],
        [20, 19],
        [19, 18],
        [18, 20],
        [12,  9],
        [ 9,  0],
        [ 0, 25],
        [25, 24],
        [24, 21]])

In [497]:
xs[:10]

tensor([[12, 21],
        [21, 20],
        [20, 20],
        [20,  9],
        [12, 18],
        [18,  3],
        [ 3, 25],
        [25,  1],
        [ 1, 25],
        [25,  9]])

In [498]:
prompt("em")

'eme..e.tieiaiuyoy..a.e...aiailaiy.iil.tiaepmai.mata.iameai.a.n.oy.oai.aaab.oiaa.iim..pbq.myii.yai.aziknra.i.ed....iiiikagiiakia..aqiii.adpiitiyaaiiioi.aa.i.izo.aoaaesi.aammmaraa.bramaamssaiioapib.yat.e.iawaliaa..sihefabaii.aa.oka.aa.odfw.taoi.bamip..aa.a.imyaihakmaiiyvmbegiiam.o.iym.iiatiagox.iiaeie.e.a.ayaebaabaaaanaaieniiaaaaieakigaeaa.as.aaoasiaira.eae.as..eai.yao..bimmefmilaz.anelai.omnaia.eaeisimainaiaoab.ai.o.ahameiey.ioobagi.eiia.ii.iibreeoag.eaiymrasaa.iaaaiet...eageioiliaii.ynayr.yaamo.if'

High regularization results in a fairly large disparity between normal and dev sets, but it seems to produce better results in odd cases

In [499]:
xenc = functional.one_hot(txs, num_classes=vocab_len).float().flatten(start_dim=1)
logits = xenc @ ws
counts = logits.exp()
probs = counts / counts.sum(1, keepdim=True)

loss = -probs[torch.arange(int(tnum/2)), tys].log().mean()
loss

tensor(2.4886, grad_fn=<NegBackward0>)

In [500]:
ws.shape

torch.Size([54, 27])

Wait

In [503]:
ws = torch.randn((vocab_len, vocab_len, vocab_len))
ws.requires_grad = True
ws.shape

torch.Size([27, 27, 27])

In [509]:
xenc = functional.one_hot(xs, num_classes=vocab_len).float()
xenc.shape

torch.Size([157152, 2, 27])

In [516]:
xenc[0].shape

torch.Size([2, 27])

In [515]:
(xenc[0] @ ws).shape

torch.Size([27, 2, 27])

Holup, gotta learn matrix multiplication again, it's hard being a 3 dimensional being trying to comprehend 700 dimensional math